In [1]:
#from  scapy.all import *
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from scapy.all import *
#files name in pcap direcoty 
import glob

In [2]:
#Take all split values from folder, that I just done above

In [3]:
split_files = glob.glob("NF_othr/*.pcap")

In [4]:
split_files

['NF_othr/1_host_2_average_1.pcap',
 'NF_othr/2_host_2_average_2.pcap',
 'NF_othr/2_host_2_elastic_2.pcap',
 'NF_othr/2_host_2_logistic_1.pcap',
 'NF_othr/2_host_2_conventional_1.pcap',
 'NF_othr/1_host_2_elastic_2.pcap',
 'NF_othr/2_host_2_exponential_2.pcap',
 'NF_othr/1_host_2_exponential_2.pcap',
 'NF_othr/1_host_2_conventional_2.pcap',
 'NF_othr/1_host_2_exponential_1.pcap',
 'NF_othr/2_host_2_exponential_1.pcap',
 'NF_othr/2_host_2_conventional_2.pcap',
 'NF_othr/2_host_2_elastic_1.pcap',
 'NF_othr/1_host_2_logistic_1.pcap',
 'NF_othr/2_host_2_average_1.pcap',
 'NF_othr/1_host_2_elastic_1.pcap',
 'NF_othr/1_host_2_conventional_1.pcap',
 'NF_othr/1_host_2_logistic_2.pcap',
 'NF_othr/2_host_2_logistic_2.pcap',
 'NF_othr/1_host_2_average_2.pcap']

In [5]:
total_split_files = len(split_files)

In [6]:
total_split_files

20

In [7]:
names = split_files[0].split('.')


In [8]:
node_number = names[0].split('/')

In [9]:
node_number[1]

'1_host_2_average_1'

In [ ]:
for s_files in range(total_split_files):
    print(split_files[s_files])
    pcap =rdpcap(split_files[s_files])
    reset= []
    for p in pcap:
        if p.haslayer(TCP) and (p[TCP].flags == 'RA' or p[TCP].flags == 'R') : #reset,ack or reset flag RA mean both reset and ack
            if p[IP].src == '192.168.0.150': #server port
                #print("2015-----")
                #print(p[TCP].sport)
                #print(p[TCP].dport)
                reset.append(p[TCP].dport)
            else:
                #print("not 2015---")
                reset.append(p[TCP].sport)
                #print(p[TCP].sport)
                #print(p[TCP].dport)
            
    #head= []

    #for p in pcap:
        #if p.haslayer(TCP): 
            #if Raw in p[TCP]:
                #raw = str(p[Raw])
                #if 'HEAD' in raw:
                    #head.append(p[TCP].sport)   

#print(head)


    connection=[]; connection_sip=[]; connection_dip=[]; connection_sport=[]; connection_sip=[]
    c=0
    pcap[1].show()  
    extra=0; segg=1; array=[]; array2=[]
    for ppp in pcap:
        # Select full connection except reset and head connection 
        if (ppp.haslayer(TCP)) and ( ppp[TCP].sport not in reset):
            #print('fuu'+str(ppp[TCP].sport))
            if Raw in ppp[TCP]:
                if(ppp[IP].dst == '192.168.0.150'):# server ip
                    if 'GET' in str(ppp[Raw]):
                        connection.append(c)
                        c=c+1
                        connection_sip.append(ppp[IP].src)
                        connection_sport.append(ppp[TCP].sport)
                        if ("sintel_enc_x264_dash.mpd" in str(ppp[Raw]) or "sintel_enc_x264_dash_init.mp4"  in str(ppp[Raw])):
                                extra=extra+1
                        #elif 'segment'+str(segg) in str(ppp[Raw]):
                         #           array.append(ppp[TCP].sport)
                          #          segg=segg+1
                        #else:
                            #array2.append(segg)
                            #segg=segg+1
                            #continue
                        #print(array)
                        #print(segg)
    #print(extra)                  
    conn=[] ;ij=extra-1;ik=0            
    conn=sorted(list(set(connection_sport)))                    
    for q in range(len(conn)-extra):
         for qq in pcap:
            if  qq.haslayer(TCP) and ((qq[TCP].sport == conn[q+extra])):
                 if Raw in qq[TCP]:
                    if(qq[IP].dst == '192.168.0.150'):# server ip
                        if 'GET' in str(qq[Raw]):
                                print(str((q+extra)-ij) )
                                if ((q+extra)-ij)>59:
                                    break
                                if 'segment'+str((q+extra)-ij) in str(qq[Raw]):
                                    array.append(qq[TCP].sport)
                                    print(str((q+extra)-ij) )
                                    print("conn:"+str(conn[(q+extra)-ij]))
                                    #print(array)
                                    segg=segg+1
                                    #ij=3
                                #print(segg)
                                #perint
                                else:
                                    ik=ik+1
                                    array2.append((q+extra)-ij)
                                    ij=extra-(1+ik)
    df= pd.DataFrame(columns=['Port','Segment','RTT','Throughput', 'Packets'])# rtt=uplink ms, th=downlink byte per sec, sp= downlink packets

    t1=0;t2=0;l=0;inner =True;k=0;b=True;s=0;

    i=1; 
    length= (len(connection))-extra
    print(extra)
    print(length)
    m=extra
    for j in range(len(conn)-m):
        pac1= True;pac= True; ttt=True;f=True; c=0;tc= True; r1=0; initrtt=0; r2=0;  inner=True; inner2=False;httprtt=0; aaa=0;summ=0;l=0;tspop=0
        for p in pcap:
            if p.haslayer(TCP) and (p[TCP].sport == conn[j+m] or p[TCP].dport== conn[j+m]):
                if pac:
                    p1_t=p.time
                    pac=False
                if(p[IP].dst == connection_sip[1]):  
                    if(  p[TCP].flags == 'SA'):
                        if(tc==True):
                            p2_t= p.time
                            initrtt= (p2_t - p1_t) * 1000
                            print("initrtt" + str(initrtt))
                            tc=False

                if  p[IP].src == connection_sip[j+m] and pac1 and  Raw in p[TCP]:
                                                    #if (segment in str(p[Raw])):
                    r1 = p.time
                    print('r1---'+ str(r1))
                    pac1 = False
                    #print(p[IP].src)

                if p[IP].dst == connection_sip[j+m]:

                        if Raw in p[TCP]:
                            l=l+int(len(p[Raw]))
                            if len(p[TCP]) > 32:
                                    pcakets=p[TCP]
                                    tstop=p.time
                                    c=c+1  

                        if ttt and (p[TCP].flags == 'A' or p[TCP].flags == 'PA'):
                            r2=p.time
                            httprtt= r2 - r1
                            print('r2---'+ str(r2))
                            aaa= httprtt * 1000
                            summ= (initrtt+aaa)/2
                            ttt=False    

                if p[TCP].flags == 'FA':# 
                    th=float(l/(tstop-r1))
                    if j+m in array2:
                        i=i+1
                        continue


                    #df.loc[i]=[aaa ,th ,c ]
                    df = df.append(pd.Series([conn[j+m], i, aaa ,th ,c], index=df.columns ), ignore_index=True)
                    i=i+1
                    #print(df)

                    break
    names = split_files[s_files].split('.')

    node_number = names[0].split('/')
        
    
    export_csv = df.to_csv ('NF_othr/qos_othr/'+node_number[1]+'.csv', index = None, header=True) #Don't forget   
    print('Done') 


NF_othr/1_host_2_average_1.pcap
###[ Ethernet ]### 
  dst       = 22:74:fc:d1:06:10
  src       = ae:f5:d5:e4:68:d9
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 57498
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xd841
     src       = 192.168.0.1
     dst       = 192.168.0.150
     \options   \
###[ TCP ]### 
        sport     = 43004
        dport     = 2015
        seq       = 946972757
        ack       = 2007455027
        dataofs   = 8
        reserved  = 0
        flags     = A
        window    = 56
        chksum    = 0x820e
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (2576767675, 2989146446))]

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
1
60
initrtt32.614000
r1---1593412

initrtt73.471000
r1---1593414190.118860
r2---1593414190.158982
initrtt49.103000
r1---1593414192.269394
r2---1593414192.308596
initrtt48.121000
r1---1593414194.615520
r2---1593414194.654899
initrtt87.776000
r1---1593414197.071559
r2---1593414197.133537
initrtt51.995000
r1---1593414199.967024
r2---1593414200.030160
initrtt82.429000
r1---1593414202.880583
r2---1593414202.942054
initrtt79.582000
r1---1593414205.090075
r2---1593414205.149442
initrtt80.746000
r1---1593414207.709783
r2---1593414207.774782
initrtt79.740000
r1---1593414210.518036
r2---1593414210.578627
initrtt44.261000
r1---1593414212.938757
r2---1593414212.969541
initrtt67.229000
r1---1593414215.848936
r2---1593414215.878565
initrtt77.919000
r1---1593414218.740761
r2---1593414218.798386
initrtt75.446000
r1---1593414221.243004
r2---1593414221.299750
initrtt52.891000
r1---1593414223.630610
r2---1593414223.649592
initrtt64.920000
r1---1593414227.076992
r2---1593414227.110001
initrtt73.324000
r1---1593414229.326987
r2---1593414229

initrtt28.114000
r1---1593412629.085666
r2---1593412629.115131
initrtt24.385000
r1---1593412630.576453
r2---1593412630.596926
initrtt34.480000
r1---1593412632.096333
r2---1593412632.115409
initrtt21.138000
r1---1593412633.693223
r2---1593412633.702641
initrtt16.422000
r1---1593412635.449748
r2---1593412635.458120
initrtt18.872000
r1---1593412637.136091
r2---1593412637.165666
initrtt17.609000
r1---1593412638.735902
r2---1593412638.753413
initrtt18.863000
r1---1593412640.314065
r2---1593412640.324391
initrtt28.523000
r1---1593412641.898949
r2---1593412641.922458
initrtt33.218000
r1---1593412643.603467
r2---1593412643.623245
initrtt15.571000
r1---1593412645.280788
r2---1593412645.295439
initrtt28.298000
r1---1593412647.015131
r2---1593412647.038199
initrtt25.880000
r1---1593412648.811075
r2---1593412648.832834
initrtt29.988000
r1---1593412650.450696
r2---1593412650.465046
initrtt25.642000
r1---1593412652.150182
r2---1593412652.171022
initrtt22.317000
r1---1593412653.877962
r2---1593412653

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
2
60
initrtt195.855000
r1---1593413189.662649
r2---1593413189.819286
initrtt367.856000
r1---1593413191.503490
r2---1593413191.803634
initrtt228.416000
r1---1593413194.564283
r2---1593413194.791051
initrtt205.344000
r1---1593413198.426237
r2---1593413198.640470
initrtt133.286000
r1---1593413202.324857
r2---1593413202.513789
initrtt159.929000
r1---1593413206.156864
r2---1593413206.315712
initrtt1110.958000
r1---1593413210.169965
r2---1593413210.246559
initrtt81.376000
r1---1593413212.391949
r2---1593413212.483125
initrtt171.752000
r1---1593413214.593249
r2---1593413214.745199
initrtt16.228000
r1---1593413217.292153
r2---1593413217.292451
initrtt143.866000
r1---1593413219.085107
r2---1593413219.228412
initrtt99.547000
r1---1593413221.322840
r2---1593413221.323000
initrtt164.075000
r1---1593413222.690198
r2---15934132

initrtt1010.002000
r1---1593413942.063539
r2---1593413942.063560
initrtt39.167000
r1---1593413942.113105
r2---1593413942.152901
initrtt63.086000
r1---1593413946.677352
r2---1593413946.738832
initrtt104.233000
r1---1593413946.858890
r2---1593413946.948685
initrtt7.003000
r1---1593413950.100437
r2---1593413950.100561
initrtt89.722000
r1---1593413950.202767
r2---1593413950.241120
initrtt7.249000
r1---1593413953.507118
r2---1593413953.507243
initrtt45.953000
r1---1593413953.561173
r2---1593413953.611951
initrtt6.812000
r1---1593413955.538798
r2---1593413955.539052
initrtt45.588000
r1---1593413955.608084
r2---1593413955.661890
initrtt6.597000
r1---1593413957.825802
r2---1593413957.825900
initrtt38.586000
r1---1593413957.872454
r2---1593413957.904653
initrtt7.323000
r1---1593413961.034708
r2---1593413961.034913
initrtt45.037000
r1---1593413961.087637
r2---1593413961.137452
initrtt54.511000
r1---1593413962.420045
r2---1593413962.459858
initrtt76.273000
r1---1593413962.551641
r2---1593413962.6

initrtt34.852000
r1---1593413994.193878
r2---1593413994.229668
initrtt41.832000
r1---1593413997.602398
r2---1593413997.635663
initrtt59.777000
r1---1593414000.089569
r2---1593414000.128159
initrtt74.968000
r1---1593414002.842848
r2---1593414002.899264
initrtt63.979000
r1---1593414005.755786
r2---1593414005.788234
initrtt55.920000
r1---1593414008.038408
r2---1593414008.075757
initrtt69.318000
r1---1593414011.092783
r2---1593414011.129133
initrtt57.046000
r1---1593414013.819671
r2---1593414013.845927
initrtt53.032000
r1---1593414015.969055
r2---1593414016.004437
initrtt49.182000
r1---1593414018.466971
r2---1593414018.484797
initrtt56.073000
r1---1593414022.214557
r2---1593414022.240513
initrtt56.286000
r1---1593414026.057209
r2---1593414026.082916
initrtt66.546000
r1---1593414030.267949
r2---1593414030.302595
initrtt68.383000
r1---1593414035.469439
r2---1593414035.520189
initrtt63.170000
r1---1593414039.451708
r2---1593414039.506020
initrtt47.639000
r1---1593414043.374852
r2---1593414043

initrtt23.562000
r1---1593411869.009461
r2---1593411869.023256
initrtt20.356000
r1---1593411872.359275
r2---1593411872.372724
initrtt29.644000
r1---1593411874.193171
r2---1593411874.215717
initrtt33.125000
r1---1593411877.415363
r2---1593411877.418473
initrtt29.092000
r1---1593411880.563571
r2---1593411880.585777
initrtt28.815000
r1---1593411882.180566
r2---1593411882.202731
initrtt26.713000
r1---1593411885.333879
r2---1593411885.353653
initrtt23.854000
r1---1593411888.007422
r2---1593411888.028003
initrtt40.772000
r1---1593411890.337650
r2---1593411890.339684
initrtt29.826000
r1---1593411893.306539
r2---1593411893.316499
initrtt24.178000
r1---1593411896.530662
r2---1593411896.551578
initrtt17.826000
r1---1593411898.525992
r2---1593411898.542121
initrtt22.733000
r1---1593411901.143334
r2---1593411901.159113
initrtt24.104000
r1---1593411903.386235
r2---1593411903.406255
initrtt14.299000
r1---1593411905.381331
r2---1593411905.388014
initrtt25.599000
r1---1593411907.459841
r2---1593411907

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
2
60
initrtt154.744000
r1---1593413649.388473
r2---1593413649.552969
initrtt306.981000
r1---1593413651.127892
r2---1593413651.435266
initrtt75.320000
r1---1593413654.022049
r2---1593413654.118190
initrtt216.200000
r1---1593413658.487450
r2---1593413658.701141
initrtt81.130000
r1---1593413662.074267
r2---1593413662.150001
initrtt69.140000
r1---1593413665.450098
r2---1593413665.516082
initrtt103.619000
r1---1593413668.168714
r2---1593413668.249757
initrtt161.009000
r1---1593413670.740082
r2---1593413670.899804
initrtt998.393000
r1---1593413674.092495
r2---1593413674.092521
initrtt100.198000
r1---1593413676.098684
r2---1593413676.174516
initrtt56.699000
r1---1593413678.301919
r2---1593413678.397273
initrtt72.759000
r1---1593413681.018801
r2---1593413681.101165
initrtt114.115000
r1---1593413682.379464
r2---1593413682.

initrtt7.750000
r1---1593413018.667863
r2---1593413018.668030
initrtt13.351000
r1---1593413020.302170
r2---1593413020.308860
initrtt13.184000
r1---1593413022.500896
r2---1593413022.501310
initrtt36.868000
r1---1593413024.761222
r2---1593413024.777191
initrtt34.661000
r1---1593413027.235121
r2---1593413027.254767
initrtt21.651000
r1---1593413029.824311
r2---1593413029.831317
initrtt25.808000
r1---1593413032.441818
r2---1593413032.459366
initrtt23.552000
r1---1593413034.639461
r2---1593413034.659863
Done
NF_othr/1_host_2_logistic_1.pcap
###[ Ethernet ]### 
  dst       = ae:35:26:87:e0:0b
  src       = d2:32:b4:10:e1:66
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 24394
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x5992
     src       = 192.168.0.1
     dst       = 192.168.0.150
     \options   \
###[ TCP ]### 
        sport     = 44312
        dport    